<a href="https://colab.research.google.com/github/LoicSteve/data-science-portfolio/blob/main/data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ingest

In [1]:
import pandas as pd

## EDA

## Modeling

## Conclusion